In [2]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd
from obspy import read
from tqdm import tqdm

2025-05-05 17:14:13.697193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746483253.711317   36866 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746483253.715398   36866 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746483253.725186   36866 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746483253.725202   36866 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746483253.725203   36866 computation_placer.cc:177] computation placer alr

#hola

In [3]:
# Forzar uso de CPU (opcional)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [5]:
# Ruta al modelo guardado
model_path = '/mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/src/models/augmented_ranges_huber_data.keras'

# Cargar el modelo entrenado
model = tf.keras.models.load_model(model_path)

2025-05-05 16:15:50.589569: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
def pad_or_trim(signal, target_length=8000):
    if len(signal) > target_length:
        return signal[:target_length]
    elif len(signal) < target_length:
        pad_width = target_length - len(signal)
        return np.pad(signal, (0, pad_width), mode='constant')
    return signal

def load_inference_data(data_path):
    raw_signals = []
    files_df = pd.read_csv(os.path.join(data_path, 'feature_files.csv'))
    
    print(f'Cargando señales desde {data_path}...')
    for file in tqdm(files_df['file']):
        file_path = os.path.join(data_path, file)
        st = read(file_path)
        signal = st[0].data
        signal = pad_or_trim(signal)      # Luego pad/trim
        raw_signals.append(signal)

    X_raw = np.array(raw_signals)
    X_raw = X_raw.reshape(X_raw.shape[0], -1, 1)

    # Cargar características wavelets
    X_wavelets = np.load(os.path.join(data_path, 'wavelet_features.npy'))

    return X_raw, X_wavelets, files_df

In [7]:
# Definir la ruta al directorio con los datos a procesar
inference_path = '/mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/procesed/used_data/testing'

# Cargar datos
X_raw, X_wavelets, files_df = load_inference_data(inference_path)

# Realizar predicciones
print('Realizando predicciones...')
predictions = model.predict([X_raw, X_wavelets])

# Crear DataFrame con resultados
results_df = pd.DataFrame({
    'file': files_df['file'],
    'predicted_time': predictions.flatten()
})

# Guardar resultados
output_path = os.path.join(os.path.dirname(inference_path), 'predictions.csv')
results_df.to_csv(output_path, index=False)
print(f'Resultados guardados en: {output_path}')

Cargando señales desde /mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/procesed/used_data/testing...


Cargando señales desde /mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/procesed/used_data/testing...


100%|██████████| 496/496 [00:26<00:00, 18.76it/s]



Cargando señales desde /mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/procesed/used_data/testing...


100%|██████████| 496/496 [00:26<00:00, 18.76it/s]



...

Realizando predicciones...


I0000 00:00:1746479809.217057   21673 service.cc:152] XLA service 0x7f1f9400f160 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746479809.217110   21673 service.cc:160]   StreamExecutor device (0): Host, Default Version
2025-05-05 16:16:49.227546: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 2/16 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step 

I0000 00:00:1746479809.469570   21673 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step
Resultados guardados en: /mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/procesed/used_data/predictions.csv
Resultados guardados en: /mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/procesed/used_data/predictions.csv


In [8]:
# Definir la ruta al directorio con los datos a procesar
inference_path = '/mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/raw/process/testing_filtered'

# Cargar datos
X_raw, X_wavelets, files_df = load_inference_data(inference_path)

# Realizar predicciones
print('Realizando predicciones...')
predictions = model.predict([X_raw, X_wavelets])

# Crear DataFrame con resultados
results_df = pd.DataFrame({
    'file': files_df['file'],
    'predicted_time': predictions.flatten()
})

# Guardar resultados
output_path = os.path.join(os.path.dirname(inference_path), 'predictions_eval.csv')
results_df.to_csv(output_path, index=False)
print(f'Resultados guardados en: {output_path}')

Cargando señales desde /mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/raw/process/testing_filtered...


Cargando señales desde /mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/raw/process/testing_filtered...


100%|██████████| 790/790 [00:27<00:00, 28.64it/s]



Cargando señales desde /mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/raw/process/testing_filtered...


100%|██████████| 790/790 [00:27<00:00, 28.64it/s]



Realizando predicciones...
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step
Resultados guardados en: /mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/raw/process/predictions_eval.csv
Resultados guardados en: /mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/raw/process/predictions_eval.csv


## Convert Predictions to UNIX Timestamps

Convert predictions to UNIX timestamps (seconds since epoch) to match training data format

In [13]:
# Read predictions file
pred_path = '/mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/raw/process/predictions_eval.csv'
predictions_df = pd.read_csv(pred_path)

# Extract date components from filenames
def extract_datetime(filename):
    # Remove .mseed extension and get date/time digits
    datetime_str = filename.replace('.mseed', '')
    
    # Extract components
    month = int(datetime_str[0:2])
    day = int(datetime_str[2:4])
    hour = int(datetime_str[4:6])
    minute = int(datetime_str[6:8])
    
    # Construct datetime - use 2024 as base year
    from datetime import datetime
    return datetime(2024, month, day, hour, minute)

# Convert to UNIX timestamps
predictions_df['base_timestamp'] = predictions_df['file'].apply(lambda x: extract_datetime(x).timestamp())
predictions_df['lec_p'] = predictions_df['base_timestamp'] + predictions_df['predicted_time']

# Create output dataframe in required format
result_df = pd.DataFrame()
result_df['file'] = predictions_df['file'].str.replace('.mseed','')
result_df['lec_p'] = predictions_df['lec_p']

# Save results
output_path = '/mnt/c/Users/Usuario/Documents/Studies/GicoProject/SeismicWaves/data/raw/process/predictions_eval_timestampsv5.csv'
result_df.to_csv(output_path, index=False)

print(f"First few predictions:")
print(result_df.head())

First few predictions:
       file         lec_p
0  01011426  1.704137e+09
1  01020023  1.704173e+09
2  01020428  1.704188e+09
3  01020441  1.704188e+09
4  01020453  1.704189e+09
